# Overview of a Ray task

## What is a Ray task ?

A Ray task is a **distributed** function in the Ray framework. **Distributed** means Ray executes the function on a different process and possibly on a separate machine from where you initiate it.

## When should you use Ray tasks?

You should consider using a Ray task in these situations:

- **Current state**:
    - Your code is in Python, Java, or C++.
    - Your code runs slowly because it runs sequentially
    - Your code is complex
        - it involves scaling functions that have different resource requirements
        - it involves dynamically composing functions together
           - i.e. functions depend on each other and require passing data between them
- **Desired state**:
    - You would like to speed up your code's execution by running it asynchronously
    - You would like to scale your code to run on multiple machines with ease
- **Desired path**:
    - You don't want to perform a major code rewrite to enable this speed up

Let's look at an example to make this more concrete.

### Example

We have a Python function conveniently named `expensive_computation`, which executes a computation that requires significant resources and time.

`expensive_computation` performs the following function calls:
1. call `perform_naive_matrix_multiplication` to perform a naive matrix multiplication
2. call `compute_matrix_sum` to return the sum of the elements in the resulting matrix

<!-- References: 
- [Matrix multiplication definition in wikipedia](https://en.wikipedia.org/wiki/Matrix_multiplication#Definition)
-->

In [ ]:
from itertools import product

Matrix = list[list[int]]

def perform_naive_matrix_multiplication(size: int) -> Matrix:
    matrix1 = matrix2 = [[1 for _ in range(size)] for _ in range(size)]

    result = [[0 for _ in range(size)] for _ in range(size)]
    for i, j, k in product(range(size), range(size), range(size)):
        result[i][j] += matrix1[i][k] * matrix2[k][j]

    return result

def compute_matrix_sum(matrix: Matrix) -> int:
    return sum([sum([col for col in row]) for row in matrix])

def expensive_computation(size: int) -> int:
    result = perform_naive_matrix_multiplication(size)
    return compute_matrix_sum(result)

We require running our `expensive_computation` for `n_runs`. We can do this sequentially, but it will take a long time.

In [ ]:
n_runs = 10
size = 300
results = [expensive_computation(size) for _ in range(n_runs)]
expected_result = size**3
assert sum(results) == n_runs * expected_result

Below is our code execution visualized 

<img src="https://assets-training.s3.us-west-2.amazonaws.com/ray-core/task-actor-lifecycle/v2/scheduling/sequential_execution_python_func_v3.svg.svg" width="600px">

#### Desired execution

We would like, instead, to execute our Python function in parallel and distribute it over as many machines as possible

<img src="https://assets-training.s3.us-west-2.amazonaws.com/ray-core/task-actor-lifecycle/v2/scheduling/desired_execution_python_func_v4.svg" width="600px">

In the next section, we will learn how to achieve this desired state using Ray tasks!

## How do you define a Ray task ?

To define a Ray task, you can use the `ray.remote` decorator in Python. When you decorate a Python function with `ray.remote`, it converts the Python function into a Ray task.

### Example
Let's revisit our example and convert our `perform_naive_matrix_multiplication` function into a Ray task.

In [ ]:
import ray

decorator = ray.remote(num_cpus=2) # decorator to convert Python function to Ray task

perform_naive_matrix_multiplication_task = decorator(perform_naive_matrix_multiplication)

In this example, we also specify the task resource requirements inside the `ray.remote` decorator. Here are some more arguments you can pass the decorator to specify your task's resource requirements:
- `num_cpus`: the quantity of CPU resources to reserve for this task. By default, tasks use 1 CPU resource
- `num_gpus`: The quantity of GPU resources to reserve for this task.  By default, this is set based on detected GPUs.
- `resources`: The quantity of various custom resources to reserve for this task (e.g. think of Google `TPU` or AWS accelerators). By default, tasks don't use any custom resources.

If we had a "proper" matrix multiplication implementation and we wanted to speed it up on a GPU we can. For now, our example will have to remain simple for educational purposes.

## How do you submit a Ray task ?

To submit a Ray task, you can use the `.remote` method instead of calling the task directly. The `.remote` method accepts the same arguments as the original function.

### Example

Let's submit our `perform_naive_matrix_multiplication` task. 

- The `.remote` method is a non-blocking method that immediately returns an object reference.
- An object reference is a placeholder reference for the value that will be returned by the Ray task (think of it as a future).
- The value is named an object given Ray makes use of a fast and efficient object store for passing data dependencies between tasks.

In [ ]:
size = 100
object_ref = perform_naive_matrix_multiplication_task.remote(size=size)
object_ref

This will return something like `ObjectRef(359ec6ce30d3ca2dffffffffffffffffffffffff0100000001000000)`

## How do you fetch the Ray task results ?

Use `ray.get(object_ref)` to wait for a given object's value to be ready.

### Example

We use `ray.get` to fetch the resulting object value from our object_ref

In [ ]:
object_value = ray.get(object_ref)
assert len(object_value[0]) == size

## How do you compose Ray tasks together ? 

You can call tasks within another task in Ray without any additional considerations

### Example 

Let's revisit our initial intention of scaling `expensive_computation`. In this case we want our `expensive_computation` task to:
- Submit the `perform_naive_matrix_multiplication` task and get back an object reference `result_obj_ref` to the result of the matrix multiplication.
- Pass the `result_obj_ref` to the `compute_matrix_sum` task
    - Conveniently Ray will resolve the underlying result of an object reference when passing data dependencies between tasks.
- Use `ray.get` to wait for the resulting object of the `compute_matrix_sum` task and return it

In [ ]:
@ray.remote 
def compute_matrix_sum(matrix: Matrix) -> int:
    return sum([sum([col for col in row]) for row in matrix])

@ray.remote
def expensive_computation(size: int) -> int:
    result_obj_ref = perform_naive_matrix_multiplication_task.remote(size)
    sum_obj_ref = compute_matrix_sum.remote(result_obj_ref)
    return ray.get(sum_obj_ref)

Now we will:
- Submit our `expensive_computation` task for `n_runs` by calling `expensive_computation.remote`
- Fetch the results by calling `ray.get(object_refs)`
    - Note that when `ray.get` is called against a "sequence of object references", it will block until the last object is available.

In [ ]:
n_runs = 10
size = 300
object_refs = [expensive_computation.remote(size) for _ in range(n_runs)]
results = ray.get(object_refs)
expected_result = size**3
assert sum(results) == n_runs * expected_result

Here is our execution visualized using Ray tasks. Do you spot the resemblance with our desired execution diagram?

<img src="https://assets-training.s3.us-west-2.amazonaws.com/ray-core/task-actor-lifecycle/v2/scheduling/overview_distributed_ray_task_v3.svg" width="700px">

## How do you scale to multiple machines?

While Ray works out of the box on your laptop, to scale Ray applications on multiple nodes you must first *deploy a Ray cluster*.

A Ray cluster is a set of worker nodes connected to a common Ray head node. Ray clusters can be fixed-size, or they may autoscale up and down according to the resources requested by applications running on the cluster.

### Where can I deploy Ray clusters?
Ray provides native cluster deployment support on the following technology stacks:

- On [AWS and GCP](https://docs.ray.io/en/latest/cluster/vms/index.html#cloud-vm-index). Community-supported Azure, Aliyun and vSphere integrations also exist.
- On Kubernetes via the officially supported [KubeRay project](https://docs.ray.io/en/latest/cluster/kubernetes/index.html#kuberay-index)
  

# Understanding Ray task execution

Let's explore in more detail how the `expensive_computation` task is executed.

Here it is laid out in steps:

1. `expensive_computation.remote(...)` is called with its inputs
2. An `expensive_computation` task is submitted for scheduling on the Ray cluster
3. Ray will autoscale the cluster to meet the the task resource requirements
4. Ray will schedule the task to run on a worker
5. The worker process will execute the task

The task will output a resulting object which can be fetched using `ray.get`

The above steps are visualized in this diagram

<img src="https://assets-training.s3.us-west-2.amazonaws.com/ray-core/task-actor-lifecycle/v2/scheduling/overview_detailed_execution_ray_task_v3.svg" width="800px">